<a href="https://colab.research.google.com/github/Sagargupta16/LeetCode_Rating_Predictor/blob/main/LC_Contest_Rating_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#Import Libraries
import concurrent.futures
import json
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import joblib
from concurrent.futures import ThreadPoolExecutor

In [23]:
# Define GraphQL Query and Headers

# GraphQL query for fetching user contest data
query = """
query userContestRankingInfo($username: String!) {
    userContestRankingHistory(username: $username) {
        attended
        rating
        ranking
        contest {
            title
        }
    }
}
"""

# Headers for the GraphQL request
headers = {"Content-Type": "application/json"}

In [47]:
session = requests.Session()

def fetch_contest_users_count(contest_title):
    url = f"https://leetcode.com/contest/api/ranking/{contest_title}/"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Failed to fetch data"}

def read_usernames_from_json(file_path, number_of_usernames=500):
    with open(file_path, 'r') as file:
        all_usernames = json.load(file)
        return all_usernames[:number_of_usernames]

def fetch_data(username):
    response = requests.post(
        "https://leetcode.com/graphql",
        headers=headers,
        json={"query": query, "variables": {"username": username}}
    )
    if response.status_code == 200:
        return response.json().get("data", {}).get("userContestRankingHistory", [])
    else:
        print(f"Error fetching data for username {username}: {response.status_code}")
        return []  # Return an empty list in case of error


In [50]:
constest_participants = {}

def process_data(contests):
    data = []
    rating=1500
    for contest in contests:
        if contest["attended"]:
            prev_rating = rating
            rating = contest["rating"]
            ranking = contest["ranking"]
            contest_title = contest["contest"]["title"]
            contest_title = contest_title.lower().replace(" ", "-")
            words = contest_title.split("-")
            first_word, last_word = words[0], words[-1]
            if first_word == "weekly" and last_word.isdigit() and int(last_word) < 58:
              contest_title = "leetcode-" + contest_title
            if contest_title == "weekly-contest-62" :
              contest_title = "weekly-contest-by-app-academy"
            if(contest_title not in constest_participants):
              x = fetch_contest_users_count(contest_title)
              if "user_num" not in x :
                break
              constest_participants[contest_title] = x["user_num"]
            total_participants = constest_participants[contest_title]
            data.append([prev_rating, ranking, total_participants, ranking / total_participants if total_participants != 0 else 0, rating - prev_rating])
    return data

def process_batch_parallel(usernames_batch, max_workers=100):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Use map to execute the function in parallel
        results = executor.map(fetch_data, usernames_batch)

    data = []
    for contests in results:
        if contests:  # Check if contests is not empty
            data.extend(process_data(contests))

    return data


usernames = read_usernames_from_json('usernames.json')

all_data = []
batch_size = 250
for i in range(0, len(usernames), batch_size):
    batch_data = process_batch_parallel(usernames[i:i + batch_size])
    all_data.extend(batch_data)
    time.sleep(1)  # Sleep to respect API rate limits


In [51]:
df = pd.DataFrame(all_data, columns=['input1', 'input2', 'input3' , 'input4' , 'output'])
print(df)
df.to_json('data.json', orient='records', lines=True)

X = df.iloc[:, :-1].values
y = df['output'].values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.save')
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

         input1  input2  input3    input4   output
0      1500.000     110   13038  0.008437  320.626
1      1820.626      45   11877  0.003789  199.425
2      2020.051      69    7926  0.008706  102.787
3      2122.838     119   13283  0.008959   73.217
4      2196.055     384   13805  0.027816   19.398
...         ...     ...     ...       ...      ...
10773  1454.323    5098   23404  0.217826   46.008
10774  1500.000     328    6631  0.049465  215.392
10775  1715.392    1027    7873  0.130446   44.149
10776  1759.541    1376   17435  0.078922   41.787
10777  1801.328    2413   26069  0.092562   28.621

[10778 rows x 5 columns]


In [52]:
# Ensure using a GPU if available
with tf.device('/device:GPU:0'):

    # Define LSTM model with cuDNN compatible configurations
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(100, activation='tanh', recurrent_activation='sigmoid', input_shape=(1, 4)),
        tf.keras.layers.Dense(1)
    ])

    # Set a custom learning rate
    learning_rate = 0.01
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32)

    # Evaluate the model
    loss = model.evaluate(X_test, y_test)
    print("Test Loss:", loss)

    # Save the model in the recommended format (.keras)
    model.save('model.keras')


Epoch 1/100
236/236 [==============================] - 5s 6ms/step - loss: 1173.8296
Epoch 2/100
236/236 [==============================] - 1s 6ms/step - loss: 579.9818
Epoch 3/100
236/236 [==============================] - 1s 5ms/step - loss: 479.4406
Epoch 4/100
236/236 [==============================] - 1s 4ms/step - loss: 408.8857
Epoch 5/100
236/236 [==============================] - 1s 4ms/step - loss: 361.1792
Epoch 6/100
236/236 [==============================] - 1s 4ms/step - loss: 336.3144
Epoch 7/100
236/236 [==============================] - 1s 4ms/step - loss: 318.9911
Epoch 8/100
236/236 [==============================] - 1s 4ms/step - loss: 306.3169
Epoch 9/100
236/236 [==============================] - 1s 4ms/step - loss: 295.9607
Epoch 10/100
236/236 [==============================] - 1s 4ms/step - loss: 289.1132
Epoch 11/100
236/236 [==============================] - 1s 4ms/step - loss: 284.5717
Epoch 12/100
236/236 [==============================] - 1s 4ms/step - los

In [ ]:
!pip install tensorflowjs

import tensorflow as tf
import tensorflowjs as tfjs

# Load your previously saved Keras model
model_path = '/content/model.keras'  # Replace with the path to your saved model
model = tf.keras.models.load_model(model_path)

# Directory where the TensorFlow.js model will be saved
output_dir = '/content/tfjs_model'

# Convert the model
tfjs.converters.save_keras_model(model, output_dir)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [53]:
import numpy as np
import tensorflow as tf
import joblib

# Load the trained model and the scaler
model = tf.keras.models.load_model('/content/model.keras')
scaler = joblib.load('/content/scaler.save')

def get_user_input():
    """
    Prompts the user to enter the required input values.
    """
    print("Enter the input values:")
    input1 = float(input("Enter your previous rating: "))
    input2 = float(input("Enter your ranking: "))
    input3 = float(input("Enter Total Participants: "))
    input4 = input2 / input3 if input3 != 0 else 0

    return np.array([[input1, input2, input3,input4]])

def make_prediction(input_data):
    """
    Normalizes the input data and makes a prediction.
    """
    input_scaled = scaler.transform(input_data)
    input_scaled = input_scaled.reshape((input_scaled.shape[0], 1, input_scaled.shape[1]))
    prediction = model.predict(input_scaled)
    return prediction[0][0]

def main():
    user_input = get_user_input()
    prediction = make_prediction(user_input)
    print(f"Predicted change in rating: {prediction}")

if __name__ == "__main__":
    main()


Enter the input values:
Enter your previous rating: 1915
Enter your ranking: 795
Enter Total Participants: 19360
1/1 [==============================] - 0s 459ms/step
Predicted change in rating: 42.08863067626953
